In [37]:
!pip install langchain langchain-experimental openai pymysql

In [38]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [ ]:
OPENAI_API_KEY = "YOUR_API_KEY"
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [40]:
host = 'localhost'
port = '3307'
username = 'root'
#password = ''
database_schema = 'csv_db 12'
mysql_uri = f"mysql+pymysql://{username}@{host}:{port}/{database_schema}"
#mysql_uri = f"mysql+pymysql://{username:password}@{host}:{port}/{database_schema}"
db = SQLDatabase.from_uri(mysql_uri, include_tables=['book2'],sample_rows_in_table_info=2)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [41]:
def retrieve_from_db(query: str) -> str:
    db_context = db_chain(query)
    db_context = db_context['result'].strip()
    return db_context

In [42]:
def generate(query: str) -> str:
    db_context = retrieve_from_db(query)
    
    system_message = """You are a Data analysis database expert specializing in  describe demographic, professional, and financial details of individuals, likely for a purpose such as risk assessment, consumer profiling, or financial analysis. 
        Your task is to answer users' questions by providing relevant insights from the dataset. 
        The database contains the following information for each movie:
        - id
        - Age
        - Experience
        - House_ownership
        - Car_ownership
        - Profession
        - City
        - State
        - Current job years
        - Current House years
        - Risk flag
        
        Example:
        
        Input:
      Which professions have the highest proportion of individuals flagged as high-risk?

Context:
In the dataset, individuals are flagged as high-risk if their Risk_Flag is 1. The analysis shows that the professions with the highest proportion of high-risk individuals are:

Software Developers - 30%
Mechanical Engineers - 25%
Civil Servants - 20%
Output:
The professions with the highest proportion of high-risk individuals are Software Developers (30%), Mechanical Engineers (25%), and Civil Servants (20%)."""

    
    human_qry_template = HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
        
        Context:
        {db_context}
        
        Output:
        """
    )
    messages = [
      SystemMessage(content=system_message),
      human_qry_template.format(human_input=query, db_context=db_context)
    ]
    response = llm(messages).content
    return response

In [43]:
generate("give me a brief description about the dataset?")



> Entering new SQLDatabaseChain chain...
give me a brief description about the dataset?
SQLQuery:SELECT `Id`, `Income`, `Age`, `Experience`, `Married`, `House_Ownership`, `Car_Ownership`, `Profession`, `CITY`, `STATE`, `CURRENT_JOB_YRS`, `CURRENT_HOUSE_YRS`, `Risk_Flag` 
FROM book2
LIMIT 5;
SQLResult: [('1', '1303834', '23', '3', 'single', 'rented', 'no', 'Mechanical_engineer', 'Rewa', 'Madhya_Pradesh', '3', '13', '0'), ('2', '7574516', '40', '10', 'single', 'rented', 'no', 'Software_Developer', 'Parbhani', 'Maharashtra', '9', '13', '0'), ('3', '3991815', '66', '4', 'married', 'rented', 'no', 'Technical_writer', 'Alappuzha', 'Kerala', '4', '10', '0'), ('4', '6256451', '41', '2', 'single', 'rented', 'yes', 'Software_Developer', 'Bhubaneswar', 'Odisha', '2', '12', '1'), ('5', '5768871', '47', '11', 'single', 'rented', 'no', 'Civil_servant', 'Tiruchirappalli[10]', 'Tamil_Nadu', '3', '14', '1')]
Answer:The dataset consists of information about individuals including their ID, income, age,

'The dataset contains information about individuals, including their ID, income, age, experience, marital status, house ownership, car ownership, profession, city, state, current job years, current house years, and risk flag.'

In [44]:
generate("what is the average income of a married person with a minimum age of 47 and above in the state Telangana?")



> Entering new SQLDatabaseChain chain...
what is the average income of a married person with a minimum age of 47 and above in the state Telangana?
SQLQuery:SELECT AVG(`Income`) AS Average_Income
FROM book2
WHERE `Married` = 'married' AND `Age` >= 47 AND `STATE` = 'Telangana'
SQLResult: [(4939952.27,)]
Answer:The average income of a married person with a minimum age of 47 and above in the state Telangana is $4,939,952.27.
> Finished chain.


'The average income of a married person with a minimum age of 47 and above in the state Telangana is $4,939,952.27.'

In [14]:
generate("According to you, to have more income, is it ok to get married or to be single? ")



> Entering new SQLDatabaseChain chain...
According to you, to have more income, is it ok to get married or to be single? 
SQLQuery:SELECT `Married`, AVG(`Income`) AS Avg_Income
FROM book2
GROUP BY `Married`
ORDER BY Avg_Income DESC
LIMIT 2;
SQLResult: [('single', 4899188.291148825), ('married', 4812740.6724213855)]
Answer:According to the data, on average, single individuals have a higher income compared to married individuals.
> Finished chain.


'On average, according to the data, single individuals have a higher income compared to married individuals.'

In [16]:
generate("what is the average risk flag of married people with age between 20 and 30?")



> Entering new SQLDatabaseChain chain...
what is the average risk flag of married people with age between 20 and 30?
SQLQuery:SELECT AVG(`Risk_Flag`) AS Average_Risk_Flag
FROM book2
WHERE `Married` = 'married' AND `Age` BETWEEN 20 AND 30
SQLResult: [(0.0,)]
Answer:The average risk flag of married people with age between 20 and 30 is 0.
> Finished chain.


'The average risk flag of married people with age between 20 and 30 is 0.'

In [18]:
generate("what are the top 3 best profession to choose as a fresher, to grow over time and have a descent income? ")



> Entering new SQLDatabaseChain chain...
what are the top 3 best profession to choose as a fresher, to grow over time and have a descent income? 
SQLQuery:SELECT `Profession`, `Income`, `Experience` 
FROM book2 
WHERE `Experience` = '3' 
ORDER BY `Income` DESC 
LIMIT 3;
SQLResult: [('Chemical_engineer', '996748', '3'), ('Librarian', '9908353', '3'), ('Financial_Analyst', '9804751', '3')]
Answer:The top 3 best professions to choose as a fresher, to grow over time, and have a decent income are Chemical Engineer, Librarian, and Financial Analyst.
> Finished chain.


'The top 3 best professions to choose as a fresher, to grow over time, and have a decent income are Chemical Engineer, Librarian, and Financial Analyst.'

In [19]:
generate("Iam a software engineer and iam not happy with my job, what profession should i choose?")



> Entering new SQLDatabaseChain chain...
Iam a software engineer and iam not happy with my job, what profession should i choose?
SQLQuery:SELECT `Profession`, COUNT(`Profession`) AS `Count`
FROM book2
WHERE `Profession` != 'Software_Developer'
GROUP BY `Profession`
ORDER BY `Count` DESC
LIMIT 1;
SQLResult: [('Fashion_Designer', 80)]
Answer:Fashion Designer
> Finished chain.


'Based on the dataset, you may consider transitioning to a profession such as Fashion Designer.'

In [23]:
generate("what state, should i move to have a low risk flag?")



> Entering new SQLDatabaseChain chain...
what state, should i move to have a low risk flag?
SQLQuery:SELECT `STATE`, COUNT(`Risk_Flag`) AS `Low_Risk_Count`
FROM book2
WHERE `Risk_Flag` = '0'
GROUP BY `STATE`
ORDER BY `Low_Risk_Count` DESC
LIMIT 5;
SQLResult: [('Uttar_Pradesh', 199), ('West_Bengal', 172), ('Andhra_Pradesh', 163), ('Maharashtra', 155), ('Bihar', 134)]
Answer:You should move to Uttar Pradesh to have a low risk flag.
> Finished chain.


'You should move to Uttar Pradesh to have a low risk flag.'

In [24]:
generate("which state retained the most house ownerships?")



> Entering new SQLDatabaseChain chain...
which state retained the most house ownerships?
SQLQuery:SELECT `STATE`, COUNT(`House_Ownership`) AS `Total_House_Ownerships`
FROM book2
WHERE `House_Ownership` = 'owned'
GROUP BY `STATE`
ORDER BY `Total_House_Ownerships` DESC
LIMIT 1;
SQLResult: [('West_Bengal', 24)]
Answer:West Bengal retained the most house ownerships with a total of 24.
> Finished chain.


'West Bengal retained the most house ownerships with a total of 24.'

In [45]:
generate("give me a brief description about the house ownerships verses the state verse the age ?")



> Entering new SQLDatabaseChain chain...
give me a brief description about the house ownerships verses the state verse the age ?
SQLQuery:SELECT `House_Ownership`, `STATE`, `Age` FROM book2 ORDER BY `House_Ownership`, `STATE`, `Age` LIMIT 5;
SQLResult: [('House_Ownership', 'STATE', 'Age'), ('norent_noown', 'Andhra_Pradesh', '50'), ('norent_noown', 'Andhra_Pradesh', '51.4659044'), ('norent_noown', 'Andhra_Pradesh', '51.5238447'), ('norent_noown', 'Andhra_Pradesh', '55')]
Answer:House ownerships vary across different states and ages. Some examples include individuals in Andhra Pradesh aged 50, 51.46, 51.52, and 55 who do not own or rent a house.
> Finished chain.


'House ownership varies across different states and ages in the dataset. For example, in Andhra Pradesh, individuals aged 50, 51.46, 51.52, and 55 do not own or rent a house. This suggests that there is a mix of house ownership statuses among individuals of different ages in Andhra Pradesh. Further analysis can provide more insights into the distribution of house ownership across states and age groups.'

In [49]:
generate("  suggest me 3 professions and 3 States that I can live happily")



> Entering new SQLDatabaseChain chain...
  suggest me 3 professions and 3 States that I can live happily
SQLQuery:SELECT DISTINCT `Profession`, `STATE` 
FROM book2 
ORDER BY RAND() 
LIMIT 3;
SQLResult: [('Technology_specialist', 'Tamil_Nadu'), ('Industrial_Engineer', 'Haryana'), ('Surgeon', 'Bihar')]
Answer:Professions: Technology specialist, Industrial Engineer, Surgeon
States: Tamil Nadu, Haryana, Bihar
> Finished chain.


'Based on the dataset, the three professions and three states where individuals are flagged as low-risk and hence potentially conducive to a happy life are:\n\nProfessions:\n1. Technology Specialist\n2. Industrial Engineer\n3. Surgeon\n\nStates:\n1. Tamil Nadu\n2. Haryana\n3. Bihar\n\nThese professions and states have a lower proportion of high-risk individuals, indicating a potentially happier and more stable living environment.'